In [3]:
import os
import os.path
import re
import itertools
import pdfkit

import pandas as pd
import numpy as np
import pymysql.cursors
import seaborn as sb
import matplotlib.pyplot as plt

from datetime import datetime

#### SELECT NEWEST CSV FILE

In [9]:
os.getcwd()
# 'c:\\Users\\ShaomengMao\\Projects\\ticket-reporter'

'c:\\Users\\ShaomengMao\\Projects\\ticket-reporter'

In [5]:
# 检查指定目录中文件名符合规则且修改时间最新的csv文件并提取修改时间

def get_latest_csv_info(directory):
    latest_file = None
    latest_mtime = 0

    # 定义匹配 csv 文件名的正则表达式
    pattern = r"案例清单列表_(\d{4})(\d{2})(\d{2})(\d{2})(\d{2})(\d{2})\.csv"
    
    # 遍历目录下的所有文件
    for filename in os.listdir(directory):
        # 使用正则表达式匹配文件名
        match = re.match(pattern, filename)
        #if filename.endswith(".csv"):
        if match:
            print(f"Compliant csv file {filename} fund.")
            filepath = os.path.join(directory, filename)
            mtime = os.path.getmtime(filepath)

            # 比较修改时间，更新最新的文件和修改时间
            if mtime > latest_mtime:
                latest_mtime = mtime
                latest_file = filename
        else:
            print(f"Not-compliant file {filename} fund.")

    # 如果找到了最新的文件
    if latest_file:
        # 获取最新文件的修改时间
        mtime = datetime.fromtimestamp(latest_mtime)
        print(mtime)

        # 将修改时间的年、月、日、时、分、秒保存在不同的变量中
        year = mtime.year
        month = mtime.month
        day = mtime.day
        hour = mtime.hour
        minute = mtime.minute
        second = mtime.second

        return latest_file, (year, month, day, hour, minute, second)
    else:
        return None, (None, None, None, None, None, None)


In [6]:
# 绝对路径
gsp_path = os.path.abspath(os.path.join(os.getcwd(), 'data\\gsp'))

latest_csv, time_stempel = get_latest_csv_info(gsp_path)
year, month, day, hour, minute, second = time_stempel

if latest_csv:
    print("最新的 CSV 文件:", latest_csv)
    print("修改时间:", year, "年", month, "月", day, "日", hour, "时", minute, "分", second, "秒")
else:
    print("目录中没有 CSV 文件")

Not-compliant file adnlsadnadas_202403071325.csv fund.
Not-compliant file arch fund.
Not-compliant file eqvasd_202403111655.txt fund.
Not-compliant file TicketList_202403071009.csv fund.
Not-compliant file TicketList_20240308172133.csv fund.
Not-compliant file udfjnafnansfasd_202302111203.csv fund.
Compliant csv file 案例清单列表_20240308172134.csv fund.
Compliant csv file 案例清单列表_20240312191412.csv fund.
Compliant csv file 案例清单列表_20240326232104.csv fund.
Compliant csv file 案例清单列表_20240327182327.csv fund.
Compliant csv file 案例清单列表_20240327230817.csv fund.
Compliant csv file 案例清单列表_20240328012218.csv fund.
Compliant csv file 案例清单列表_20240421214309.csv fund.
2024-04-21 15:47:24.044729
最新的 CSV 文件: 案例清单列表_20240421214309.csv
修改时间: 2024 年 4 月 21 日 15 时 47 分 24 秒


In [7]:
# 使用绝对路径定位最新的csv数据
path_latest_file = os.path.join(os.getcwd(), 'data\\gsp', latest_csv)
#print(path_latest_file)
# 使用 pandas 的 read_csv 函数读取 CSV 文件内容并转换为 DataFrame
tickets = pd.read_csv(path_latest_file)
print("总条目数量:", tickets.shape[0])

# 计算空值的数量
null_count = tickets['产品型号/Product Model'].isna().sum()
# 计算总行数
total_rows = len(tickets)
# 计算空值占比
null_percentage = (null_count / total_rows) * 100
# 打印结果
print("空值数量:", null_count)
print("空值占比: {:.2f}%".format(null_percentage))

# 将 NaN 值填充为 ''
tickets.fillna('', inplace=True)
# 筛选出DC相关的Tickets
tickets_dc = tickets[(tickets['产品型号/Product Model'].str.contains('IDC', case=False))]


print("IDC相关的条目数量:", tickets_dc.shape[0])

C:\Users\ShaomengMao\AppData\Local\Temp\ipykernel_32216\837442969.py:5: DtypeWarning: Columns (18,36,37,39,49) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets = pd.read_csv(path_latest_file)


总条目数量: 110977
空值数量: 80858
空值占比: 72.86%
IDC相关的条目数量: 71


#### SELECT DC CHARGER RELATED TICKETS

In [7]:
# 计算空值的数量
null_count = tickets['产品型号/Product Model'].isna().sum()
# 计算总行数
total_rows = len(tickets)
# 计算空值占比
null_percentage = (null_count / total_rows) * 100
# 打印结果
print("空值数量:", null_count)
print("空值占比: {:.2f}%".format(null_percentage))

# 将 NaN 值填充为 ''
tickets.fillna('', inplace=True)
# 筛选出DC相关的Tickets
tickets_dc = tickets[(tickets['产品型号/Product Model'].str.contains('IDC', case=False))]


print("IDC相关的条目数量:", tickets_dc.shape[0])

空值数量: 80858
空值占比: 72.86%
IDC相关的条目数量: 71


In [1]:
tickets_dc.info()

NameError: name 'tickets_dc' is not defined

In [ ]:
tickets_dc[tickets_dc['序列号/Serial Number'] == 'A2412211326']

#### EXTRACT TICKET INFO

In [ ]:

# 选取部分列
columns = ['服务案例号/Ticket No.',
            '发生日期/Create Date',
            '服务类型/Service Type',
            '序列号/Serial Number',
            '产品型号/Product Model',
            '物料代码/Product Code',
            '失效现象描述/Failure Description',
            '现场环境描述/On-Site Environment Description',
            '服务响应及处置/Service Response & Solution',
            '关联服务案例/Related Ticket',
            '信息记录/Detailed Description',
            '项目名称/Project Name',
            '客户名称/Customer Name',
            '国家/Country/Region',
            '城市/City',
            '优先级/Priority',
            '负责人/Person in Charge',
            '案例状态/Ticket Status']

# 列名转化为全英文
tickets_dc_ = tickets_dc.loc[:, columns].rename(columns=lambda x: x.split('/')[1].strip())
tickets_dc_.info()

In [ ]:
target_path = os.path.join(os.getcwd(), 'data', 'dc_tickets')
target_path

In [ ]:
filtered_df['Project No.'] = filtered_df.apply(lambda x: str(x.name + 1).zfill(4), axis=1)

#### EXTRACT PROJECT INFO

In [ ]:
project_columns = ['项目名称/Project Name',
                   '发生日期/Create Date',
                   '客户名称/Customer Name',
                   '销售合同编号/Sales Contract No.',
                   '地址/Project Street Address',
                   '城市/City',
                   '国家/Country/Region',
                   '经度/Longitude',
                   '纬度/Latitude',]

In [ ]:
local_file_name = "data.xls"

def read_local_excel(fn=local_file_name):
    # path
    file = os.getcwd() + "\\" + fn
    # 初始化空DataFrame
    df = pd.DataFrame()
    # 读取全表（直接dict中获取各个表？？？）
    reader = pd.read_excel(file, sheet_name=None)  # dict of DataFrame
    # 生成sheets表
    sheets = []
    for k in reader.keys():
        sheets.append(k)
    # 循环 sheets
    for sheet in sheets:
        # 提取 sheet 为 DataFrame
        df_sheet = pd.read_excel(file, sheet_name=sheet)
        # 删除含空值的行
        df_sheet.dropna(axis=0, how="any", inplace=True)
        # 删除原生索引列
        df_sheet = df_sheet.drop(["ID"], axis=1)
        # 添加到df
        df = pd.concat([df, df_sheet], ignore_index=True)

    return df

In [ ]:
import markdown2
import pdfkit

# 自定义Markdown内容
markdown_content = """
# My Custom Markdown Report

This is a sample Markdown report generated using Python.

## DataFrame Content

| Name    | Age | Salary |
|---------|-----|--------|
| Alice   | 25  | 50000  |
| Bob     | 30  | 60000  |
| Charlie | 35  | 70000  |
"""

# 将Markdown转换为HTML
html_content = markdown2.markdown(markdown_content)

# 将HTML内容保存到文件
html_file_path = "report.html"
with open(html_file_path, "w") as html_file:
    html_file.write(html_content)

# 将HTML文件转换为PDF
pdf_file_path = "report.pdf"
pdfkit.from_file(html_file_path, pdf_file_path)

print(f"PDF已生成：{pdf_file_path}")